In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()
# do something to prove it works
spark.sql('SELECT "Test" as c1').show()

spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(spark.sparkContext)
df = sqlContext.read.csv(input_filename)
df.write.parquet(output_path)

+----+
|  c1|
+----+
|Test|
+----+



In [3]:
print(spark.sparkContext)

<SparkContext master=local[*] appName=SimpleApp>


In [4]:
print(spark.catalog)

In [1]:
# type(sc)
type(spark)
# List sc's attributes
# dir(sc)

In [2]:
# help(sc)
print (sc.version)

In [3]:
#dbutils.fs.mount(
#  source = "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net",
#  mount_point = "/mnt/<mount-name>",
#  extra_configs = {"<conf-key>":dbutils.secrets.get(scope = "<scope-name>", key = "<key-name>")})

# Use Approach 3 from https://tsmatz.github.io/azure-databricks-exercise/exercise01-blob.html ie
# --------------------------------------------------------------------------------------------------------
#1 Create AAD AppReg/(Service Principle Account SPA name = pdtest rg app reg, client-id = 1f46a8fc-b197-41b0-942d-5aa15bf482f7) with ClientSecret (I called mine pdtest-appreg-secret) 
#2 Immediately put the Secret into a Key vault as a secret (I called mine vault-pdtest-appreg-secret), because only get one choice to view/copy that SPA secret
#3 Give that SPA "Storage Blob Data Contributor" role in IAM for Storage Acct (could also give whole RG permissions etc)
#4 Go to https://ukwest.azuredatabricks.net/?o=8385949688125694#secrets/createScope and create a Scope (I called mine SPA) for the Key vault in 2)

#sp_secret = dbutils.secrets.get(scope = "SPA", key = "vault-pdtest-appreg-secret")

#configs = {"fs.azure.account.auth.type": "OAuth",
#           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#           "fs.azure.account.oauth2.client.id": "1f46a8fc-b197-41b0-942d-5aa15bf482f7",
#           "fs.azure.account.oauth2.client.secret": sp_secret,
#           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/microsoft.onmicrosoft.com/oauth2/token"}
#           #"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/microsoft.onmicrosoft.com/oauth2/token"}
# abfss is az blob (extra s for secure) while wasbs is web based

dbutils.fs.mount(
  source = "abfss://land-reg@pdstdgpv1lrs.blob.core.windows.net",
  mount_point = "/mnt/land-reg",
  extra_configs = {"fs.azure.account.key.pdstdgpv1lrs.blob.core.windows.net":dbutils.secrets.get(scope = "SPA", key = "vault-pdtest-appreg-secret2")})
# error IllegalArgumentException: The String is not a valid Base64-encoded string

# spark.conf.set("fs.azure.account.key.pdstdgpv1lrs.blob.core.windows.net",  "<storage account key>")

# dbutils.fs.unmount("/mnt/<mount-name>")

In [4]:
#code for going direct
# dfFilenames = dbutils.fs.ls("wasbs://land-reg@pdstdgpv1lrs.blob.core.windows.net/FirstLandRegFile.json")
dfj = spark.read.json("wasbs://land-reg@pdstdgpv1lrs.blob.core.windows.net/FirstLandRegFile.json")
display(dfj) #display is an action therefore creates spark job

import pyspark.sql.functions as f
json_schema = spark.read.json(dfj.rdd.map(lambda row: row.json)).schema
dfj.withColumn('json', f.from_json(f.col('json'), json_schema))

In [5]:
df = spark.read.csv("wasbs://land-misc@pdstdgpv1lrs.blob.core.windows.net/open_postcode_geo_TINY.csv.gz")
display(df) #display is an action therefore creates spark job

In [6]:
# display(df.schema.names)
df.printSchema()

In [7]:
df1 = df.filter("_c6 = 'Scotland' and _c5 != 1").select("_c0","_c1").distinct()

In [8]:
host = "pddemodw.database.windows.net:1433"
db   = "demo_db"
user = "pdtest_sa@pddemodw"
pwd  = dbutils.secrets.get(scope = "SPA", key = "vault-rg-pdtest-sqlsa")


#url = "jdbc:sqlserver://$host;database=$db;user=$user;password=$pwd;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
url = "jdbc:sqlserver://" + host + ";database=" + db + ";user=" + user + ";password=" + pwd + ";encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
# print(url)

jdbcDF = spark.read.jdbc(url, "land.postcode_stage")


In [9]:
jdbcDF.count()

In [10]:
# Explain plan with no column selection returns all columns
#dfSE1 = spark.read.jdbc(url, "land.postcode_stage").select("postcode").where("outcode = 'AB1'")
spark.read.jdbc(url, "land.postcode_stage").select("postcode").where("outcode = 'AB1'").explain()

# display(dfSE1)
# dfSE1.count()

In [11]:
spark.read.jdbc(url, "(select postcode from land.postcode_stage where outcode = 'AB1') v").explain()

In [12]:
df1.write.jdbc(url, "scotland")